In [1]:
import random
from time import sleep
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime


driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\ramir\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


Hites

In [2]:
url = "https://www.hites.com/tecnologia/computacion/notebook/?start=0&sz=70"
response = requests.get(url)
data = response.text
soup = BeautifulSoup(data,"html.parser")
bloque = soup.find_all("div",{"class":"col-6 col-lg-4 px-0"})

In [3]:
nombres = list()
precioaux = list()
precio = list()
codigos = list()
modelo = list()
encontrarIndiceModelo = list()
encontrarindice = list()
caracteristica = list()
almacenamientoaux = list()
almacenamiento = list()
procesadoraux = list()
procesador = list()
ramaux = list()
ram = list()
precios = list()

for i in bloque:
    nombre = i.find("a",{"class":"link product-name"})
    caract = i.find_all("li",{"class":"attribute-values"})
    precios = i.find("span",{"class":"value"})
    codigo = i.find("span",{"class":"d-block product-sku"})
    nombres.append(nombre.text)
    caracteristica.append(caract)  
    precioaux.append(precios.text)
    codigos.append(codigo.text)

""""Obtener cada elemento de almacenamiento, memoria ram y procesador definiendo un valor auxiliar 
    para almacenamiento en caso de no encontrarse en la pagina"""
for i in range(len(caracteristica)):
    if (len(caracteristica[i])==2):
        almacenamientoaux.append(caracteristica[i][0])
        ramaux.append(caracteristica[i][0])
        procesadoraux.append(caracteristica[i][1])
        
    else:
        almacenamientoaux.append(caracteristica[i][0])
        ramaux.append(caracteristica[i][1])
        procesadoraux.append(caracteristica[i][2])

#Insertar elementos de procesador y memoria ram en las listas como texto, en el caso de almacenamiento como str
for i in range(len(almacenamientoaux)):
    if(type(almacenamientoaux[i]==str)):
        almacenamiento.append(almacenamientoaux[i])
        procesador.append(procesadoraux[i].text)
        ram.append(ramaux[i].text)
        
    else:
        almacenamiento.append(almacenamientoaux[i].text)
        procesador.append(procesadoraux[i].text)
        ram.append(ramaux[i].text)  

#Modificar los elementos de almacenamiento a formato text
for i in range(len(almacenamiento)):
    almacenamiento[i] = almacenamiento[i].text
    

#Modicar valor auxiliar de almacenamiento en caso de que no se encontrará en la pagina          
for i in range(len(almacenamiento)):
    if ('RAM' in almacenamiento[i])==True:
        almacenamiento[i]="Almacenamiento no especificado"
        
#Reemplazar los saltos de linea del formato text a expacios en blanco para obtener solo los datos deseados
for i in range(len(caracteristica)):
    almacenamiento[i] = almacenamiento[i].replace('\n','')
    ram[i] = ram[i].replace('\n','')
    procesador[i] = procesador[i].replace('\n','')

#Encontrar el indice del primer slash para poder separar el nombre original sin los componentes
for i in range(len(nombres)):
    encontrarindice.append(nombres[i].find("/"))

#Dejar solo el nombre de los notebooks dentro de la lista
for i in range(len(nombres)):
    nombres[i] = nombres[i][0:encontrarindice[i]-1]
    
#Eliminación de caracteres especiales
for i in range(len(precioaux)):
    precioaux[i] = precioaux[i].replace("Price reduced from", "")
    precioaux[i] = precioaux[i].replace("\n","")
    precioaux[i] = precioaux[i].replace("to","")
    precioaux[i] = precioaux[i].replace("$","")
    precioaux[i] = precioaux[i].replace(".","")
    precioaux[i] = precioaux[i].replace("Normal","")
    precioaux[i] = precioaux[i].replace("Oferta","")
    precio.append(precioaux[i])
    
for i in range(len(almacenamiento)):
    almacenamiento[i] = almacenamiento[i].replace("Almacenamiento:", "")
    
for i in range(len(procesador)):
    procesador[i] = procesador[i].replace("Procesador:", "")
    
for i in range(len(ram)):
    ram[i] = ram[i].replace("Memoria RAM:", "")

for i in range(len(codigos)):
    codigos[i] = codigos[i].replace("Código: ", "")
    codigos[i] = codigos[i].replace("\n","")  
    
#Encontrar el indice para el modelo

for i in range(len(nombres)):
    encontrarIndiceModelo.append(nombres[i].find(" "))
    
for i in range(len(nombres)):
    modelo.append(nombres[i][encontrarIndiceModelo[i]:len(nombres[i])])
    
for i in range(len(modelo)):
    modelo[i] = modelo[i][1:len(nombres)]

In [4]:

df = pd.read_csv('NotebooksOficial.csv')
x = datetime.today().strftime('%Y-%m-%d')


In [5]:
for i in range(len(caracteristica)):
    df = df.append({'Nombre': nombres[i], 'Codigo':codigos[i], 'Modelo':modelo[i] ,'Almacenamiento':almacenamiento[i],'Procesador':procesador[i], 'Memoria Ram':ram[i], 'Precio':precio[i], 'Fecha':x},ignore_index=True)

PC FACTORY

In [6]:
url_base=[]
url_notebooks=[]
url1 = "https://www.pcfactory.cl/notebooks?categoria=735&papa=636"
url2 = "https://www.pcfactory.cl/notebooks?categoria=735&papa=636&pagina=2"

url_base.append(url1)
url_base.append(url2)


for url in url_base:
    driver.get(url)
    notebook = driver.find_elements_by_xpath('//div[@class="product"]')
    for i in notebook:
        link = i.find_element_by_tag_name('a').get_attribute('href')
        url_notebooks.append(link)


In [7]:
marca = []
nombres = []
nombre_equipo = []
modelo = []
procesador = []
almacenamiento = []
codigos = []
ram = []
precios = []

for i in url_notebooks:
    maux = 0
    moaux = 0
    almaux = 0
    proaux = 0
    ramaux = 0
    try: 
        driver.get(i)
        bloque_codigos = driver.find_elements_by_xpath('//p[@class="link color-dark"]')

        codigo = bloque_codigos[1]
        codigos.append(codigo.text)

        precio = driver.find_element_by_xpath('//div[@class="price-xl color-primary-1"]').text
        precios.append(precio)

        
        
        caracteristicas = driver.find_elements_by_xpath("//li")

        for i in range(len(caracteristicas)):
            caracteristicas[i] = caracteristicas[i].text

        for i in caracteristicas:
            if(i.find("Marca")!=-1):
                maux = maux + 1
                nombre_marca = i
            if(i.find("Modelo")!=-1):
                moaux = moaux + 1
                nombre_modelo = i
            if(i.find("Almacenamiento")!=-1):
                almaux = almaux + 1
                nombre_almacenamiento = i
            if(i.find("RAM:")!=-1 or i.find("Memoria:")!=-1 or i.find("Memoria Total:")!=-1):
                ramaux = ramaux + 1
                nombre_ram = i
            if(i.find("Procesador:")!=-1):
                proaux = proaux + 1
                nombre_procesador = i

        if(maux >= 1):
            marca.append(nombre_marca)
        elif(maux == 0):
            marca.append("Marca no encontrada")
        if(moaux >= 1):
            modelo.append(nombre_modelo)
        elif(moaux == 0):
            modelo.append("Modelo no encontrado")
        if(almaux >= 1):
            almacenamiento.append(nombre_almacenamiento)
        elif (almaux == 0 ):
            almacenamiento.append("Almacenamiento no encontrado")
        if(ramaux >= 1):
            ram.append(nombre_ram)
        elif(ramaux == 0):
            ram.append("Memoria ram no encontrada")
        if(proaux >= 1):
            procesador.append(nombre_procesador)
        elif(proaux == 0):
            procesador.append("Procesador no encontrado")

        sleep(random.uniform(2,4))
    except:
        x = 2


for i in range(len(almacenamiento)):
    almacenamiento[i] = almacenamiento[i].replace("\xa0", " ")
    almacenamiento[i] = almacenamiento[i].replace("Almacenamiento:", "")
    almacenamiento[i] = almacenamiento[i].lstrip()

for i in range(len(marca)):
    marca[i] = marca[i].replace("\xa0"," ")
    marca[i] = marca[i].replace("Marca:","")
    marca[i] = marca[i].lstrip()

for i in range(len(modelo)):
    modelo[i] = modelo[i].replace("\xa0", " ")
    modelo[i] = modelo[i].replace("Modelo:", "")
    modelo[i] = modelo[i].lstrip()

for i in range(len(ram)):
    ram[i] = ram[i].replace("\xa0", " ")
    ram[i] = ram[i].replace("Memoria RAM:", "")
    ram[i] = ram[i].replace("Memoria:", "")
    ram[i] = ram[i].replace("Memoria Total:", "")
    ram[i] = ram[i].lstrip()

for i in range(len(procesador)):
    procesador[i] = procesador[i].replace("\xa0"," ")
    procesador[i] = procesador[i].replace("Procesador:","")
    procesador[i] = procesador[i].lstrip()

for i in range(len(precios)):
    precios[i] = precios[i].replace(" ","")
    precios[i] = precios[i].replace("$","")
    precios[i] = precios[i].replace(".","")

for i in range(len(marca)):
    nombres.append(marca[i] + " " + modelo[i])

for i in range(len(codigos)):
    codigos[i] = codigos[i].replace("ID ","")

x = datetime.today().strftime('%Y-%m-%d')

for i in range(len(nombres)):
    df = df.append({'Nombre': nombres[i],'Codigo':codigos[i],'Modelo':modelo[i] ,'Almacenamiento':almacenamiento[i],'Procesador':procesador[i], 'Memoria Ram':ram[i], "Precio":precios[i], 'Fecha':x},ignore_index=True)

Falabella

In [8]:
##Obtención de urls de cada una de las pagina del sitio web para la navegación de notebooks
url_base=[]
for i in range(1,11):
    url = "https://www.falabella.com/falabella-cl/category/cat70057/Notebooks?page="+str(i)
    url_base.append(url)


##Obtención de urls de cada uno de los notebooks del sitio para la posterior navegación en cada uno de ellos
url_notebooks=[]
for url in url_base:
    driver.get(url)
    notebook = driver.find_elements_by_xpath('//div[@class="jsx-4001457643 search-results-list"]')
    for i in notebook:
        link = i.find_element_by_tag_name('a').get_attribute('href')
        url_notebooks.append(link)


marca = []
nombre_equipo = []
modelo = []
procesador = []
almacenamiento = []
codigo = []
ram = []
precios = []
for i in url_notebooks:
    nombre = []
    valor = []
    driver.get(i)
    sleep(2)
    try:
        boton = driver.find_element_by_xpath('//button[@class="jsx-3459521287 mkp-swatchButton mkp-swatchButton-collapseButton"]').click()
    except:
        boton = driver.find_element_by_xpath('//button[@class="jsx-3459521287 mkp-swatchButton mkp-swatchButton-collapseButton"]').click()
    sleep(random.uniform(4,5))
    nombre_aux = driver.find_element_by_xpath('//div[@class="jsx-3686231685 product-name fa--product-name"]').text
    elementos = driver.find_elements_by_xpath('//tr[@class="jsx-428502957"]')
    try:
        precio = driver.find_element_by_xpath('//span[@class="copy12 primary  jsx-340449923 normal    "]').text
        
    except:
        try:
            precio = driver.find_element_by_xpath('//span[@class="copy10 primary  jsx-340449923 normal    "]').text
        except:
            try:
                precio = driver.find_element_by_xpath('//h2[@class="jsx-2034229452 title6 mkp-secondary    "]').text
            except:
                try:
                    precio = driver.find_element_by_xpath('//span[@class="copy12 primary  jsx-2612542277 normal     "]').text
                except:
                    precio = driver.find_element_by_xpath('//span[@class="copy10 primary  jsx-2612542277 normal     "]').text
    codigos = driver.find_element_by_xpath('//span[@class="jsx-3408573263"]').text
    for j in elementos:
        
        nombres = j.find_element_by_xpath('.//td[@class="jsx-428502957 property-name"]').text
        valores = j.find_element_by_xpath('.//td[@class="jsx-428502957 property-value"]').text
        nombre.append(nombres)
        valor.append(valores)
        
    if ("Nombre comercial" in nombre)==True:
        index_modelo = nombre.index("Nombre comercial")
        modelo.append(valor[index_modelo].upper())
    
    if ("Nombre comercial" in nombre)==False:
        modelo.append("Modelo no encontrado")
    
    if("Procesador" in nombre)==True:
        index_procesador = nombre.index("Procesador")
        procesador.append(valor[index_procesador].upper())
        
    if("Procesador" in nombre)==False:
        procesador.append("Procesador no encontrado")
        
    if("Unidad de estado sólido SSD" in nombre)==True:
        index_almacenamiento = nombre.index("Unidad de estado sólido SSD")
        almacenamiento.append(valor[index_almacenamiento].upper())
        
    if("Unidad de estado sólido SSD" in nombre)==False:
        almacenamiento.append("Almacenamiento no encontrado")
        
    if("Memoria RAM" in nombre)==True:
        index_ram= nombre.index("Memoria RAM")
        ram.append(valor[index_ram].upper())
        
    if("Memoria RAM" in nombre)==False:
        ram.append("Memoria Ram no encontrada") 
        
    
    precios.append(precio)
    codigo.append(codigos)
    nombre_equipo.append(nombre_aux)
        

for i in range(len(precios)):
    precios[i] = precios[i].replace(" ", "")
    precios[i] = precios[i].replace("$", "")
    precios[i] = precios[i].replace(".", "")

for i in range(len(codigo)):
    codigo[i] = codigo[i].replace("Código del producto: ","")


x = datetime.today().strftime('%Y-%m-%d')
for i in range(len(url_notebooks)):
    df = df.append({'Nombre': nombre_equipo[i],'Codigo':codigo[i],'Modelo':modelo[i] ,'Almacenamiento':almacenamiento[i],'Procesador':procesador[i], 'Memoria Ram':ram[i], "Precio":precios[i], "Fecha":x},ignore_index=True)

HP

In [9]:
##Obtención de urls de cada una de las pagina del sitio web para la navegación de notebooks
url_base=[]
for i in range(1,6):
    url = "https://www.hp.com/cl-es/shop/notebooks/notebooks-intel.html?gclsrc=aw.ds&p="+str(i)
    url_base.append(url)

##Obtención de urls de cada uno de los notebooks del sitio para la posterior navegación en cada uno de ellos
url_notebooks=[]
for url in url_base:
    driver.get(url)
    notebook = driver.find_elements_by_xpath('//li[@class="item product product-item"]')
    for i in notebook:
        try:
            link = i.find_element_by_tag_name('a').get_attribute('href')
        except:
            x = 2
        url_notebooks.append(link)

##Listas que almacenaran datos encontrados dentro del sitio web
marca = []
nombre_equipo = []
modelo = []
procesador = []
almacenamiento = []
codigo = []
ram = []
precios = []

for i in url_notebooks:
    nombres = []
    valores = []
    sleep(2)
    driver.get(i)
    try:
        boton = driver.find_element_by_xpath('//div[@class="data item title" and @id="tab-label-additional"]').click()
    except:
        try:

            boton = driver.find_element_by_xpath('//a[@class="data switch" and @data-toggle="trigger"').click()
        except:
            x = "boton no encontrado"
    sleep(5)
    try:
        name = driver.find_element_by_xpath('//span[@class="base" and @data-ui-id="page-title-wrapper"]').text
    except:
        name = "Nombre no encontrado"
    try:
        price = driver.find_element_by_xpath('//span[@class="price"]').text
    except:
        price = "Precio no encontrado"
    
    try:
        code = driver.find_element_by_xpath('//div[@class="value " and @itemprop="sku"]').text
    except:
        code = "Codigo no encontrado"
    bloque = driver.find_elements_by_xpath('.//tr[@class="item"]')
    for j in bloque:
        nombre = j.find_element_by_xpath('.//th[@class="col label"]').text
        valor = j.find_element_by_xpath('.//td[@class="col data"]').text
        nombres.append(nombre)
        valores.append(valor)
        
    if ("Procesador" in nombres)==True:
        index_procesador = nombres.index("Procesador")
        procesador.append(valores[index_procesador])
        
    elif ("Procesador" in nombres)==False:
        procesador.append("Procesador no especificado")
        
    if ("Memoria, estándar" in nombres)==True:
        index_ram = nombres.index("Memoria, estándar")
        ram.append(valores[index_ram])
        
    elif ("Memoria, estándar" in nombres)==False:
        ram.append("Memoria ram no especificada")
        
    if("Descripción del disco duro" in nombres)==True:
        index_almacenamiento = nombres.index("Descripción del disco duro")
        almacenamiento.append(valores[index_almacenamiento])
    
    elif("Descripción del disco duro" in nombres)==False:
        almacenamiento.append("Almacenamiento no especificado")     
    
    
        
    nombre_equipo.append(name)
    precios.append(price)
    codigo.append(code)

for i in range(len(precios)):
    precios[i] = precios[i].replace(" ", "")
    precios[i] = precios[i].replace("$", "")
    precios[i] = precios[i].replace(".", "")
    if(precios[i]==""):
        precios[i] = "Precio no especificado"
        
for i in range(len(nombre_equipo)):
    modelo.append(nombre_equipo[i].replace("Notebook HP", ""))
    

In [10]:
#Llenado de cada uno de los campos
x = datetime.today().strftime('%Y-%m-%d')
for i in range(len(url_notebooks)):
    df = df.append({'Nombre': nombre_equipo[i], 'Codigo':codigo[i], 'Modelo':modelo[i],'Almacenamiento':almacenamiento[i],'Procesador':procesador[i], 'Memoria Ram':ram[i], 'Precio':precios[i], "Fecha":x},ignore_index=True)

Ripley


In [11]:
##Obtención de urls de cada una de las pagina del sitio web para la navegación de notebooks
url_base=[]
for i in range(1,8):
    url = "https://simple.ripley.cl/tecno/computacion/notebooks?page="+str(i)
    url_base.append(url)

##Obtención de urls de cada uno de los notebooks del sitio para la posterior navegación en cada uno de ellos
url_notebooks=[]
for url in url_base:
    driver.get(url)
    notebook = driver.find_elements_by_xpath('//div[@class="catalog-product-item catalog-product-item__container col-xs-6 col-sm-6 col-md-4 col-lg-4"]')
    for i in notebook:
        link = i.find_element_by_tag_name('a').get_attribute('href')
        url_notebooks.append(link)

##Obtención de datos dentro de cada una de las páginas del sitio web, almacenados en listas auxiliares

nombre_equipo = []
precios = []
almacenamiento = []
procesador = []
ram = []
modelo = []
codigos = []

for i in url_notebooks:
    nombres = []
    valores = []        
    driver.get(i)
    sleep(5)
    try:
        boton = driver.find_element_by_xpath('//div[@aria-controls="panel-Especificaciones"]').click()
    except:
        boton = "holi"
    sleep(2)
    try:
        ver = driver.find_element_by_xpath('//button[@class="btn-loading  rpl-show-more btn-outline-grey "]').click()
    except:
        x=2
    bloque = driver.find_elements_by_xpath('//tr')
    bloque_nombre = driver.find_elements_by_xpath('//section[@class="product-header hidden-xs"]')
    try:
        precio = driver.find_element_by_xpath('//dt[@class="product-price"]')
    except:
        try:
            precio = driver.find_element_by_xpath('//dt[@class="product-price product-price__line-thru"]')
        except:
            precio = "Precio no encontrado"
    for i in bloque_nombre:
        nombre = i.find_element_by_xpath('.//h1')
        codigo = i.find_element_by_xpath('.//span[@class="sku sku-value"]')
    for i in bloque:
        elementos = i.find_elements_by_xpath('.//td')
        for j in range(len(elementos)):
            if (j==0):
                nombres.append(elementos[j].text)
            if (j==1):
                valores.append(elementos[j].text)
                
    if ("Tipo Procesador" in nombres)==True:
        index_procesador = nombres.index("Tipo Procesador")
        procesador.append(valores[index_procesador])

    if ("Tipo Procesador" in nombres)==False:
        procesador.append("Procesador no especificado")

    if ("Memoria Ram" in nombres)==True:
        index_ram = nombres.index("Memoria Ram")        
        ram.append(valores[index_ram])
        
    if ("Memoria Ram" in nombres)==False:
        ram.append("Memoria Ram no especificada")
        
    if ("Capacidad Disco Duro" in nombres)==True:
        idex_almacenamiento = nombres.index("Capacidad Disco Duro")
        almacenamiento.append(valores[idex_almacenamiento])
        
    if ("Capacidad Disco Duro" in nombres)==False:
        almacenamiento.append("Almacenamiento no especificado")
        
    if ("Modelo" in nombres)==True:
        index_modelo = nombres.index("Modelo")
        modelo.append(valores[index_modelo])
        
    if ("Modelo" in nombres)==False:
        modelo.append("Modelo no especificado")
        
    codigos.append(codigo.text)
    try:
        precios.append(precio.text)

    except: 
        precios.append(precio)
    nombre_equipo.append(nombre.text)


for i in range(len(precios)):
    precios[i] = precios[i].replace(" ", "")
    precios[i] = precios[i].replace("$", "")
    precios[i] = precios[i].replace(".", "")

x = datetime.today().strftime('%Y-%m-%d')
for i in range(len(nombre_equipo)):
    df = df.append({'Nombre': nombre_equipo[i], 'Codigo':codigos[i],'Modelo':modelo[i],'Almacenamiento':almacenamiento[i],'Procesador':procesador[i], 'Memoria Ram':ram[i], "Precio":precios[i], "Fecha":x},ignore_index=True)

In [12]:
df= df.drop(['Unnamed: 0'], axis=1)
df.to_csv('NotebooksOficial.csv')


In [13]:
df

,Codigo,Memoria Ram,Fecha,Precio,Almacenamiento,Nombre,Modelo,Procesador
0,857128001,4 Gb,9/30/2021,299990,128 Gb,Notebook Reacondicionado Hp 14-fq0013dx,Reacondicionado Hp 14-fq0013dx,Amd Athlon
1,866158001,8 Gb,9/30/2021,549990,256 Gb,Notebook Lenovo Ideapad 3 14alc6,Lenovo Ideapad 3 14alc6,Amd Ryzen 5
2,861708001,8 Gb,9/30/2021,769990,512 Gb,Notebook Gamer Asus Tuf Gaming Fx506lh-hn082t,Gamer Asus Tuf Gaming Fx506lh-hn082t,Intel Core I5
3,841656001,8 Gb,9/30/2021,399990,1 Tb,Notebook Lenovo Ideapad 3,Lenovo Ideapad 3,Amd Athlon
4,861707001,4 Gb,9/30/2021,419990,128 Gb,Notebook Asus X515ea-bq968t,Asus X515ea-bq968t,Intel Core I3
...,...,...,...,...,...,...,...,...
3846,1951589,Memoria Ram no especificada,2021-10-03,499990,Almacenamiento no especificado,NOTEBOOK HP 240 G7 N4020 4GB 500GB WIN10H + SO...,240 G7,Procesador no especificado
3847,2523377,Memoria Ram no especificada,2021-10-03,1424598,Almacenamiento no especificado,"HP PROBOOK 445 G8, AMD RYZEN7-5800U, 512 GB SS...",445 G8,Procesador no especificado
3848,2234473,8GB,2021-10-03,349900,500 GB,HP 240 G7 CELERON N4020 8GB 500GB SIN SISTEMA ...,1D0F9LT-M,Intel Celeron
3849,2367595,8GB,2021-10-03,591990,256 GB,NOTEBOOK LENOVO IDEAPAD 3 15.6' - BLUE,Lenovo Ideapad 3 15.6'' - Touch Blue,Intel Core i3
